In [22]:
participant_id = "P1"

In [23]:
%load_ext autoreload
%autoreload 2
VIDEOS_FOLDER = "./data/raw/videos"
import importlib.util
from pathlib import Path
import os
analyzer_paths = Path("./src/utils").resolve()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
print(analyzer_paths)

/Users/bassel27/Downloads/hireverse/src/pipelines/src/utils


In [25]:

print(analyzer_paths)
spec_facial= importlib.util.spec_from_file_location("face_analyzer", os.path.join(analyzer_paths,"face_analyzer.py"))
face_analyzer = importlib.util.module_from_spec(spec_facial)
spec_facial.loader.exec_module(face_analyzer)

/Users/bassel27/Downloads/hireverse/src/pipelines/src/utils


FileNotFoundError: [Errno 2] No such file or directory: '/Users/bassel27/Downloads/hireverse/src/pipelines/src/utils/face_analyzer.py'

In [ ]:

spec_lexical=importlib.util.spec_from_file_location("lexical_analyzer", os.path.join(analyzer_paths, "lexical_analyser.py"))
lexical_analyser = importlib.util.module_from_spec(spec_lexical)
spec_lexical.loader.exec_module(lexical_analyser)
# Use it like a module


In [ ]:
face_analyzer = face_analyzer.FaceAnalyzer()

frames = face_analyzer.get_video_frames_for_participant(
    participant_id, VIDEOS_FOLDER, num_selected_frames=5)

# Lexical Features

In [ ]:

#audio_file_path = "./data/raw/audio/Trimmed_P1.wav"

# Create an instance of the lexicalanalyser class
#Lexicalanalyser = lexical_analyser.lexicalanalyser(audio_file_path)
# Extract all features
#features = Lexicalanalyser.extract_all_features()

# Print the extracted features
#print(features)

# Facial Features

### Face Mesh

In [ ]:
for frame in frames:
    detected_faces_landmarks = face_analyzer.process_image_results(frame.image)
    frame.facial_landmarks_obj= face_analyzer.get_largest_face_landmarks_obj(frame.image, detected_faces_landmarks)
    if frame.facial_landmarks_obj:
        frame.facial_landmarks = frame.facial_landmarks_obj.landmark

### Face

In [ ]:

for frame in frames:
    if frame.facial_landmarks:
        frame.face = face_analyzer.get_face_coordinates(frame.facial_landmarks, frame.image)

### Smile

In [ ]:
import numpy as np


SMOOTH_WINDOW = 5
happiness_buffer = []
def smooth_happiness(happiness_prob):
    if happiness_prob is None:
        return 0 # TODO: change?
    happiness_buffer.append(happiness_prob)
    if len(happiness_buffer) > SMOOTH_WINDOW:
        happiness_buffer.pop(0)
    return np.mean(happiness_buffer)


for i, frame in enumerate(frames):
    face_roi = face_analyzer.get_face_roi_image(frame.image, frame.face, expand_ratio=1.1)
    frame.smile = smooth_happiness(face_analyzer.get_smile_from_frame(face_roi))

### Selected Facial Features

In [ ]:
for frame in frames:
    frame.two_landmarks_connectors = face_analyzer.get_selected_facial_landmarks(frame.facial_landmarks)

### Head Pose

In [ ]:

texts = []
for frame in frames:
    result = face_analyzer.get_face_angles(frame.image, frame.facial_landmarks)
    frame.face_angles = result

# Prosodic Extraction

In [ ]:
spec_prosody = importlib.util.spec_from_file_location("prosody_analyzer", os.path.join(analyzer_paths, "prosody_analyzer.py"))
prosody_analyzer_module = importlib.util.module_from_spec(spec_prosody)
spec_prosody.loader.exec_module(prosody_analyzer_module)

ProsodicFeatures = prosody_analyzer_module.ProsodicFeatures
ProsodyAnalyzer = prosody_analyzer_module.ProsodyAnalyzer


In [ ]:
from models.model_features import ProsodicFeatures
from src.utils.prosody_analyzer import ProsodyAnalyzer
 

prosody_analyzer = ProsodyAnalyzer(participant_id)
prosodic_features: ProsodicFeatures = prosody_analyzer.extract_all_features()

# Features Storage

### Facial Features Aggregation

In [ ]:
from src.utils.feature_storage import FeatureStorage

feature_storage = FeatureStorage("./data/processed/interview_features.csv")
facial_features = feature_storage.aggregate_facial_features(frames)

In [ ]:

feature_storage.save_to_csv(participant_id, prosodic_features, facial_features)

# Display Frames

In [ ]:
for frame in frames:
    frame.reset_drawable_image()
    # frame.draw_face_border()
    
    frame.draw_selected_facial_landmarks(draw_lines=True)
    
    frame.put_face_angles()
    # frame.draw_facial_landmarks()
    frame.display()